# Some example demographies

We illustrate some advanced options of `momi.Demography()`.

In [1]:
from momi import Demography, to_ms_cmd

### Specifying demographies in different time scales

Users can choose to input demographies in ms time scale, the more 'standard' coalescent time scale, or in per-generation units.

In [2]:
# a simple Demography, in ms units
events = [('-eg', 0.0, 'a', 1.0), ('-ej', .5, 'a', 'b'), ('-en', 1., 'b', .5)]
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3))

print "Demography in ms syntax:"
print to_ms_cmd(demo)

Demography in ms syntax:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [3]:
# the same Demography, but specified in the 'standard' (not 'ms') time scale
# this means that there are 2*N_ref generations per unit time (instead of 4*N_ref as in ms)

# notice times are 2x longer in the 'standard' units.
# as consequence, growth rate per unit time is 2x slower
events = [('-eg', 0.0, 'a', 0.5), ('-ej', 1., 'a', 'b'), ('-en', 2., 'b', .5)]

# use time_scale='standard' to specify standard coalescent units
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3), time_scale='standard')

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [4]:
# the same Demography, but specified in per-generation units
# we'll assume a diploid population size of 10,000

# join event 5e3 generations ago. population size drops to 5000 at 1e4 generations ago.
# growth rate per generation is 1e-4 in population 'a'
events = [('-eg',0.0,'a',1e-4),('-ej', 5e3, 'a', 'b'), ('-en', 1e4, 'b', 5000)]

# use default_N to set the diploid population size
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3), default_N=10000)

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


### Ghost populations

A 'ghost' population is one that is not present in the sample.

`momi` syntax is more flexible than `ms` about ghost populations. In particular, ghost populations can be immediately affected by demographic events.

By contrast, in `ms`, ghost populations cannot be modified by events until they have been "created" with the **-es** flag.

In [5]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0), # can set growth rate of 'z' at t=0, even tho its a ghost population
          ('-ep', .5, 'a', 'z', .25), # pulse migration from z to a
          ('-ej', 1.0, 'z', 'a')]

demo = Demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the growth event (-eg) doesn't happen until the admixture event (-es)

Demography in ms syntax.
-I 1 10 -es 0.5 1 0.75 -en 0.5 2 0.606530659713 -eg 0.5 2 1.0 -ej 1.0 2 1


In `momi`, we can also have sampled populations merge **-ej** with ghost populations, even if there is no admixture event.

In [6]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0),
          ('-ej', .5, 'a', 'z'), # 'z' is empty, so this is equivalent to just changing size/growth of 'a'
          ('-en', 1.0, 'z', 1.0)]

demo = Demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the -ej event doesn't appear; instead just the size and growth rate of 'a' change

Demography in ms syntax.
-I 1 10 -en 0.5 1 0.606530659713 -eg 0.5 1 1.0 -en 1.0 1 1.0


### Ancient DNA

Use the `sampled_t` parameter to indicate that a population was sampled in the past.

In [7]:
events = [('-ej', 1.0, 'a', 'b')]

# the samples from 'b' are ancient DNA, from time t=0.5
demo = Demography(events, ['a','b'], (6, 10), sampled_t=[0.0, .5])

try:
    print to_ms_cmd(demo)
except:
    print "Cannot convert demography with archaic populations to ms syntax"

Cannot convert demography with archaic populations to ms syntax


### Demography with a lot of parameters

An example on how to programatically construct a demography with a lot of parameters (too many to conveniently write down by hand).

In [8]:
# piecewise constant demography, with 50 pieces

import random

n_pieces = 50
t = 0.0
events = []
for i in range(n_pieces):
    t = t + random.expovariate(100.)
    N = random.expovariate(1.0)
    events.append(('-en', t, 0, N))

demo = Demography(events, [0], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)

Demography in ms syntax.
-I 1 10 -en 0.00481037470035 1 0.785723073805 -en 0.00895512492658 1 0.97582753493 -en 0.0232264495176 1 0.260369464169 -en 0.024345671839 1 0.53366195552 -en 0.031569398418 1 4.66442758898 -en 0.0334062551903 1 0.622435003017 -en 0.0528760492345 1 0.912090979766 -en 0.126734371245 1 0.976752457602 -en 0.131959218781 1 0.199479083269 -en 0.135271503079 1 0.301441043923 -en 0.137000986452 1 1.53486289382 -en 0.14483033857 1 0.883932112702 -en 0.15522884469 1 0.089656312611 -en 0.177734904858 1 1.09668591882 -en 0.182913893799 1 0.264890714317 -en 0.183972160996 1 1.28309080472 -en 0.190973574278 1 2.4790409442 -en 0.198075216938 1 0.664141573454 -en 0.210567726219 1 5.20928065928 -en 0.210791184085 1 2.23890298554 -en 0.228646848759 1 0.0436317214433 -en 0.241165248808 1 0.927817106854 -en 0.243489220105 1 1.86735921566 -en 0.245137819296 1 0.557195049058 -en 0.24560268439 1 2.40426782852 -en 0.246695827956 1 1.00321006852 -en 0.257282614045 1 0.565723481867 -en